In [5]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/anaconda/envs/azureml_py310_sdkv2/bin/python
3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)


In [6]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute
import os
from azure.ai.ml.entities import Environment
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

In [7]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


# Intro

Para el tuneo de hiperparámetros para un problema de clasificación o regresión, se crea el modelo utilizando librerias de ML como ser pytorch, scikit-learn, tensorflow, keras,etc. Luego, se configura los hiperparámetros propios del modelo a ser tuneados. 
En este caso, utilizaremos una máquina de vectores de soporte (SVM). Es un modelo de aprendizaje automático muy potente y versátil, capaz de realizar clasificación, regresión e incluso detección de valores atípicos lineales o no lineales. 

Para el dataset de `weatherAUS_ML.csv` tunearemos 2 hiperparámetros: 
- `min_samples_leaf`: The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.

- `criterion` : The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “log_loss” and “entropy” both for the Shannon information gain, see Mathematical formulation.

Nota: El dataset fue previamente limpiado y esta almacenado en el siguiente repositorio: 

https://github.com/sharonmaygua/rain_prediction

# Crear un cluster

Se utiliza el nombre de `cpu-cluster`, la familia `STANDARD_DS3_V2` y `4` instancias dedicadas.

In [10]:
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
except Exception:
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )

    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

# Crear un entorno personalizado

Esto nos va a permitir que las instancias de cluster no tengan problema ejecutando el código de `src/train.py`. El archivo `./env/conda.yaml` contiene todas las dependencias necesarias que se instalaran en las instancias.

In [11]:
env_name = 'sklearn-env'
job_env = Environment(
    name=env_name,
    description="sklearn 0.24.2",
    conda_file='./env/conda.yaml',
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
job_env = ml_client.environments.create_or_update(job_env)


# Ejecutar el código en el cluster

Este comando nos va a permitir correr el script `./src/train.py` en la instancia. Se hace referencia al dataset que se va a usar que está previamente cargado en el Datastorage de azure. También, como es solamente una instancia que va a correr, se le pasan los parámetros necesarios utilizados por el script, que son `min_samples_split` y `criterion`.

In [14]:
job = command(
    code="./src",
    command="python train.py --dataset_path ${{inputs.dataset_path}} --min_samples_leaf ${{inputs.min_samples_leaf}} --criterion ${{inputs.criterion}}",
    environment=f"{job_env.name}:{job_env.version}",
    experiment_name='rain-prediction-exp-decision-tree-hyp',
    display_name="rain-prediction-exp-decision-tree-hyp",
    inputs={
        "dataset_path": Input(
            type="uri_file",
            path='https://raw.githubusercontent.com/sharonmaygua/rain_prediction/main/weatherAUS_ML.csv',
        ),
        "min_samples_leaf": 3,
        "criterion": "entropy",
    },
    compute=cpu_compute_target,
)


In [15]:
returned_job = ml_client.jobs.create_or_update(job)

In [16]:
ml_client.jobs.stream(returned_job.name)

RunId: willing_lettuce_k3xfqvt0t3
Web View: https://ml.azure.com/runs/willing_lettuce_k3xfqvt0t3?wsid=/subscriptions/3deaa453-5a6c-4bcd-85f1-1645c3ccd539/resourcegroups/diplomado_ucb/workspaces/rain_prediction_aus

Execution Summary
RunId: willing_lettuce_k3xfqvt0t3
Web View: https://ml.azure.com/runs/willing_lettuce_k3xfqvt0t3?wsid=/subscriptions/3deaa453-5a6c-4bcd-85f1-1645c3ccd539/resourcegroups/diplomado_ucb/workspaces/rain_prediction_aus



# Tuneo de hiperparámetros

En este caso, se reusa el job creado anteriormente pero como una función que configura el espacio de estados de búsqueda para los dos hiperparámetros. Luego se utiliza el método `sweep` para configurar cómo se desea hacer la búsqueda.

In [17]:
job_for_sweep = job(
    min_samples_leaf=Choice([1,3,7,9]),
    criterion=Choice(values=['entropy','gini']),
)

sweep_job = job_for_sweep.sweep(
    compute=cpu_compute_target,
    sampling_algorithm="random",
    primary_metric="F1 Score",
    goal="Maximize",
    max_total_trials=12,
    max_concurrent_trials=4,
)

returned_sweep_job = ml_client.create_or_update(sweep_job)
ml_client.jobs.stream(returned_sweep_job.name)

RunId: ivory_ant_xq0h9qrk85
Web View: https://ml.azure.com/runs/ivory_ant_xq0h9qrk85?wsid=/subscriptions/3deaa453-5a6c-4bcd-85f1-1645c3ccd539/resourcegroups/diplomado_ucb/workspaces/rain_prediction_aus

Streaming azureml-logs/hyperdrive.txt

[2023-10-30T01:11:24.575417][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-10-30T01:11:25.1215064Z][SCHEDULER][INFO]Scheduling job, id='ivory_ant_xq0h9qrk85_0' 
[2023-10-30T01:11:25.3859095Z][SCHEDULER][INFO]Scheduling job, id='ivory_ant_xq0h9qrk85_1' 
[2023-10-30T01:11:25.4946178Z][SCHEDULER][INFO]Scheduling job, id='ivory_ant_xq0h9qrk85_2' 
[2023-10-30T01:11:25.5406437Z][SCHEDULER][INFO]Successfully scheduled a job. Id='ivory_ant_xq0h9qrk85_0' 
[2023-10-30T01:11:25.5466457Z][SCHEDULER][INFO]Scheduling job, id='ivory_ant_xq0h9qrk85_3' 
[2023-10-30T01:11:25.488472][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-10-30T01:11:25.7163571Z][SCHEDULER][INFO]Success

In [22]:
import shutil
shutil.make_archive('.', 'zip', './aldunatelipa')

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/aldunatelipac1/code/Users/aldunatelipac/..zip'